In [1]:
import pandas as pd
import numpy as np
import torch
import csv
from sentence_transformers import SentenceTransformer, util
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as func
from torch.nn import Linear as lin
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import string
from nltk import word_tokenize
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import spacy
from torch.distributions.multinomial import Multinomial
from transformers import BertForTokenClassification
from transformers import PreTrainedTokenizer
from transformers import AutoTokenizer
from transformers import DistilBertTokenizer, DistilBertForTokenClassification

/home/henry/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-31 20:09:10.577970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 20:09:10.578429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 20:09:10.578560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
df = pd.read_csv("Train_Tagged_Titles.tsv.gz", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)

In [4]:
def get_text(df):
    text = []
    for i in range(1,5001):
        listing = df[df["Record Number"] == str(i)]
        text.append(listing["Token"].values.tolist())

    return text

In [ ]:
get_text(df[:10])

In [5]:
last = None
for i in range(len(df)):
    if type(df["Tag"][i]) == float:
        df["Tag"][i] = last
    last = df["Tag"].values[i]
    

In [6]:
def get_labels(df, idx, t):
    out = [-100]
    listing = df[df["Record Number"] == str(idx + 1)]

    for idx, row in listing.iterrows():
        out += (t(row["Token"], padding='max_length', max_length = 512, truncation=True, return_tensors="pt")["input_ids"].argmin().item() - 2)*[tags[row["Tag"]]]
    out += [-100]*(512 - len(out))
    return out

In [ ]:

total_data = None
for i in range(1,5001):
    if i == 1:
        total_data = df[df["Record Number"] == str(i)]
    else:
        total_data.append(df[df["Record Number"] == str(i)])
    


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
a = tokenizer(get_text(df), padding='max_length', max_length=512, is_split_into_words=True, truncation=True, return_tensors="pt")
labels = []
for i in range(len(a)):
    labels.append(get_labels(df, i, tokenizer))
labels = torch.tensor(labels)
a.token_type_ids = labels
    


In [7]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=32)
model.to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [19]:
class DataSequence(torch.utils.data.Dataset):

    def __init__(self, d):
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        a = tokenizer(get_text(d), padding='max_length', max_length=512, is_split_into_words=True, truncation=True, return_tensors="pt")
        labels = []
        for i in range(len(a)):
            labels.append(get_labels(d, i, tokenizer))
        labels = torch.tensor(labels)
        for k in a.keys():
            a[k].to(torch.int32)
        self.encodings = a
        self.labels = labels.to(torch.int32)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(torch.int32)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
class DataSequence1(torch.utils.data.Dataset):

    def __init__(self, d):
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        a = tokenizer(get_text(d), padding='max_length', max_length=512, is_split_into_words=True, truncation=True, return_tensors="pt")
        labels = []
        for i in range(len(a)):
            labels.append(get_labels(d, i, tokenizer))
        labels = torch.tensor(labels)
        for k in a.keys():
            a[k].to(torch.int32)
        self.encodings = a
        self.labels = labels.to(torch.int32)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(torch.int32)
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [32]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_eval_batch_size=100, num_train_epochs=100,disable_tqdm=False)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
d1 = DataSequence(df)
d2 = DataSequence(df)
trainer = Trainer(model=model,args=training_args,train_dataset=d1,eval_dataset=d2)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/henry/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/reso

In [33]:
trainer = Trainer(model=model,args=training_args,train_dataset=d1,eval_dataset=d2)

In [34]:
trainer.train()

***** Running training *****
  Num examples = 3
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"

/tmp/ipykernel_15659/4138526111.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_15659/4138526111.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx]).to(torch.int32).type(torch.LongTensor)

***** Running Eva

{'eval_loss': 0.000525879324413836, 'eval_runtime': 0.0832, 'eval_samples_per_second': 36.071, 'eval_steps_per_second': 12.024, 'epoch': 1.0}



***** Running Evaluation *****
  Num examples = 3
  Batch size = 100


                                             


                            
                                            

 33%|███▎      | 1/3 [27:33<00:00,  2.88it/s]  






{'eval_loss': 0.000518342771101743, 'eval_runtime': 0.0809, 'eval_samples_per_second': 37.071, 'eval_steps_per_second': 12.357, 'epoch': 2.0}



***** Running Evaluation *****
  Num examples = 3
  Batch size = 100


                                             


                            
                                            

 33%|███▎      | 1/3 [27:34<00:00,  2.88it/s]  






...


***** Running Evaluation *****
  Num examples = 3
  Batch size = 100


                                             


                            
                                            

 33%|███▎      | 1/3 [28:11<00:00,  2.88it/s]   






{'eval_loss': 0.00017345225205644965, 'eval_runtime': 0.0863, 'eval_samples_per_second': 34.764, 'eval_steps_per_second': 11.588, 'epoch': 99.0}



***** Running Evaluation *****
  Num examples = 3
  Batch size = 100


                                             


                            
                                            

 33%|███▎      | 1/3 [28:11<00:00,  2.88it/s]    






Training completed. Do not forget to share your model on huggingface.co/models =)


                                             
                                            

100%|██████████| 100/100 [00:39<00:00,  2.56it/s]

{'eval_loss': 0.00017342840146739036, 'eval_runtime': 0.0874, 'eval_samples_per_second': 34.343, 'eval_steps_per_second': 11.448, 'epoch': 100.0}
{'train_runtime': 39.1386, 'train_samples_per_second': 7.665, 'train_steps_per_second': 2.555, 'train_loss': 0.00043888162821531296, 'epoch': 100.0}


TrainOutput(global_step=100, training_loss=0.00043888162821531296, metrics={'train_runtime': 39.1386, 'train_samples_per_second': 7.665, 'train_steps_per_second': 2.555, 'train_loss': 0.00043888162821531296, 'epoch': 100.0})

In [52]:
a.label_ids

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2 │
│ 47 in __getattr__                                                                         │
│                                                                                           │
│    244 │                                                                                  │
│    245 │   def __getattr__(self, item: str):                                              │
│    246 │   │   try:                                                                       │
│ ❱  247 │   │   │   return self.data[item]                                                 │
│    248 │   │   except KeyError:                                                           │
│    249 │   │   │   raise AttributeError                                                   │
│    250                                                                                    │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'label_ids'

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_3567/2221106546.py:1 in <cell line: 1>                                     │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3567/2221106546.py'                  │
│ /home/henry/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2 │
│ 49 in __getattr__                                                                         │
│                                                                                           │
│    246 │   │   try:                                                                       │
│    247 │   │   │   return self.data[item]                                                 │
│    248 │   │   except KeyError:                                                           │
│ ❱  249 │   │   │   raise AttributeError                                                   │
│    250 │                                                                                  │
│    251 │   def __getstate__(self):                                                        │
│    252 │   │   return {"data": self.data, "encodings": self._encodings}                   │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError

In [61]:
class DataSequence(torch.utils.data.Dataset):

    def __init__(self, d):
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        a = tokenizer(get_text(d), padding='max_length', max_length=512, is_split_into_words=True, truncation=True, return_tensors="pt")
        labels = []
        for i in range(len(a)):
            labels.append(get_labels(d, i, tokenizer))
        labels = torch.tensor(labels)
        tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.texts = a
        self.labels = labels

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)
        return batch_data, batch_labels

In [13]:
tags = {}
cnt = 0
for i in set(df["Tag"].values.tolist()):
    cnt += 1
    tags[i] = cnt

In [39]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = DistilBertForTokenClassification.from_pretrained('distilbert-base-uncased', num_labels=len(tags))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [40]:
from tqdm import tqdm

In [60]:
torch.cuda.memory_allocated()/1024/1024/1024

1.2204227447509766

In [46]:
def train_loop(model, df):
    df_train, df_val = train_test_split(df, train_size=0.8)
    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, num_workers=1, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, num_workers=1, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = torch.optim.Adam([[v for v in model.parameters()][-1]], lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        print(f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')
torch.cuda.empty_cache()
LEARNING_RATE = 0.005
EPOCHS = 5
BATCH_SIZE = 2
model = BertModel()
train_loop(model,total_data)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/henry/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_3567/747853408.py:78 in <cell line: 78>                                    │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3567/747853408.py'                   │
│ /tmp/ipykernel_3567/747853408.py:27 in train_loop                                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3567/747853408.py'                   │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/tqdm/std.py:1195 in __iter__              │
│                                                                                           │
│   1192 │   │   time = self._time                                                          │
│   1193 │   │                                                                              │
│   1194 │   │   try:                                                                       │
│ ❱ 1195 │   │   │   for obj in iterable:                                                   │
│   1196 │   │   │   │   yield obj                                                          │
│   1197 │   │   │   │   # Update and possibly print the progressbar.                       │
│   1198 │   │   │   │   # Note: does not call self.update(1) for speed optimisation.       │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:681 in     │
│ __next__                                                                                  │
│                                                                                           │
│    678 │   │   │   if self._sampler_iter is None:                                         │
│    679 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)            │
│    680 │   │   │   │   self._reset()  # type: ignore[call-arg]                            │
│ ❱  681 │   │   │   data = self._next_data()                                               │
│    682 │   │   │   self._num_yielded += 1                                                 │
│    683 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                   │
│    684 │   │   │   │   │   self._IterableDataset_len_called is not None and \             │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:1376 in    │
│ _next_data                                                                                │
│                                                                                           │
│   1373 │   │   │   │   self._task_info[idx] += (data,)                                    │
│   1374 │   │   │   else:                                                                  │
│   1375 │   │   │   │   del self._task_info[idx]                                           │
│ ❱ 1376 │   │   │   │   return self._process_data(data)                                    │
│   1377 │                                                                                  │
│   1378 │   def _try_put_index(self):                                                      │
│   1379 │   │   assert self._tasks_outstanding < self._prefetch_factor * self._num_workers │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:1402 in    │
│ _process_data                                     